# Typesafe plotting?
One idea of how to "avoid vega" was to see if we could make this "typesafe". Somehow. If you had a bunch of case classes which followed the vega JSON structure, then you'd get tab completion, editor support etc. Maybe this would dodge the "having to learn vega" problem.

It turns out, that vega publish a [schema](https://github.com/vega/schema). What you'd now need, is a tool which [turned JSON schema into case classes](https://quicktype.io) along with someone nutty enough to fork it, [implement dodgy, experimental scala 3 support](https://github.com/Quafadas/quicktype/blob/scala3/src/quicktype-core/language/Scala3.ts) and point it at the vega schema. Then you'd get some [code]() and [javadoc]() along these lines, and could try this;

In [3]:
import $ivy.`io.github.quafadas::dedav4s:0.8.1`

import viz.PlotTargets.almond
import viz.dsl.vegaLite.*
import viz.dsl.DslPlot.*

trait XYPlottable extends Selectable:
  val x: Double
  val y: Double

trait LinePlottable extends XYPlottable

case class LinePlottable1(xVar:Double, yVar:Double) extends LinePlottable {
  val x = xVar
  val y = yVar
}

extension [P <: XYPlottable](p: Seq[P])
  def toPlotDataset: URLData = 
    val inlineDataset : InlineDataset = for(p1 <- p) yield {
        //enc.encodeObject(p1).toMap.mapValues(_.some).toMap
        Map("x"-> Encoder.encodeDouble(p1.x).some, "y" -> Encoder.encodeDouble(p1.y ).some)
      }
    URLData(
      values = inlineDataset.some
    )

// extension ( xyPlottable: Seq[LinePlottable1] )(using pt: PlotTarget)
//   def linePlot = 
//     val data = xyPlottable.toPlotDataset.some
//     val plotDef = VegaLiteDsl(
//       $schema = "https://vega.github.io/schema/vega-lite/v5.json".some,
//       description = "a Test".some,
//       data = data,
//       mark = "line".some,
//       encoding = EdEncoding(
//         x = XClass(field = "x".some, `type`= "quantitative".some).some,
//         y = YClass(field = "y".some, `type`= "quantitative".some).some
//       ).some,
//       height = "container".some,
//       width = "container".some
//     )
//     plotDef.plot(using pt)

// val interesting = Mysterious(1.0,2.0, "hi")  
// val plotMe = Seq(    
//   NotPlottable(1.0,2.0),
//   NotPlottable(2.5,3.5)
// ).barPlot()

exception caught when loading class $: java.lang.AssertionError: assertion failed


: 

In [21]:
plotElementDist("https://google.com")
//val plot = plotElementDist("https://google.com")
// show(plot.spec) // for debugging a troublesome spec

res20: PieChart = PieChart(
  mods = List(
    ammonite.$sess.cmd17$Helper$$Lambda$3590/0x00000008016f2e68@7ec1f41b,
    ammonite.$sess.cmd17$Helper$$Lambda$3591/0x00000008016f3268@136d214f,
    ammonite.$sess.cmd17$Helper$$Lambda$3592/0x00000008016f3668@31d934b8
  )
)

In [20]:
plotElementDist("https://www.scala-lang.org")

res19: PieChart = PieChart(
  mods = List(
    ammonite.$sess.cmd17$Helper$$Lambda$3590/0x00000008016f2e68@7ec1f41b,
    ammonite.$sess.cmd17$Helper$$Lambda$3591/0x00000008016f3268@136d214f,
    ammonite.$sess.cmd17$Helper$$Lambda$3592/0x00000008016f3668@335ba5df
  )
)

I quite like this, because once the setup problem is solved, the "intent" is, to my eye, rather readable. Whilst plotting the html element counts of popular websites is... pointless, this should extend nicely to other, helpful problems :-).

## Elephant

There is however, a rather large elephant in the room - for this to work "for you", you have to be willing to do the legwork of learning vega. Whilst easy enough to trace the spec back, and find the "example" this refers to... you'll need to understand and ultimately innovate on vega. 

That is actually a pretty fun undertaking... but it's (at least) a multiple day time investment. (A worthwhile one, I claim!)